In [67]:
import torch
import torch.nn as nn

### Tools for data processing 

In [68]:
import os
import time
import math
from collections import Counter
import pprint
pp = pprint.PrettyPrinter(indent=1)

In [69]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []
        self.counter = {}
        self.total = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
            self.counter.setdefault(word, 0)
        self.counter[word] += 1
        self.total += 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)

In [70]:
class Corpus(object):
    def __init__(self, path):
        # We create an object Dictionary associated to Corpus
        self.dictionary = Dictionary()
        # We go through all files, adding all words to the dictionary
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))
        
    def tokenize(self, path):
        """Tokenizes a text file, knowing the dictionary, in order to tranform it into a list of indexes"""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens, incr = 0, 0
            for line in f:
                words = ['<sos>'] + line.split() + ['<eos1>']*(incr % 2 == 0) + ['<eos2>']*(incr % 2 == 1)
                incr +=1
                for word in words:
                    self.dictionary.add_word(word)
                tokens += len(words)
        
        # Once done, go through the file a second time and fill a Torch Tensor with the associated indexes 
        with open(path, 'r') as f:
            ids = torch.LongTensor(tokens)
            token, incr = 0, 0
            for line in f:
                words = ['<sos>'] + line.split() + ['<eos1>']*(incr % 2 == 0) + ['<eos2>']*(incr % 2 == 1)
                incr +=1
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1
        return ids

In [71]:
###############################################################################
# Load data
###############################################################################

data = './corpus/'
corpus = Corpus(data)

In [72]:
print(corpus.dictionary.total)
print(len(corpus.dictionary.idx2word))
print(len(corpus.dictionary.word2idx))

print(corpus.train.shape)
print(corpus.train[0:7])
print([corpus.dictionary.idx2word[corpus.train[i]] for i in range(40)])

print(corpus.valid.shape)
print(corpus.valid[0:7])
print([corpus.dictionary.idx2word[corpus.valid[i]] for i in range(7)])

2020173
29669
29669
torch.Size([1518729])
tensor([0, 1, 2, 3, 4, 5, 6])
['<sos>', 'Impatients', 'désirs', 'd’', 'une', 'illustre', 'vengeance', '<eos1>', '<sos>', 'Dont', 'la', 'mort', 'de', 'mon', 'père', 'a', 'formé', 'la', 'naissance', ',', '<eos2>', '<sos>', 'Enfants', 'impétueux', 'de', 'mon', 'ressentiment', '<eos1>', '<sos>', 'Que', 'ma', 'douleur', 'séduite', 'embrasse', 'aveuglément', ',', '<eos2>', '<sos>', 'Vous', 'régnez']
torch.Size([366404])
tensor([   0,   44,  145,  146,   64,  731, 1053])
['<sos>', 'Et', 'c’', 'est', 'à', 'faire', 'enfin']


In [73]:
# We now have data under a very long list of indexes: the text is as one sequence.
# The idea now is to create batches from this. Note that this is absolutely not the best
# way to proceed with large quantities of data (where we'll try not to store huge tensors
# in memory but read them from file as we go) !
# Here, we are looking for simplicity and efficiency with regards to computation time.
# That is why we will ignore sentence separations and treat the data as one long stream that
# we will cut arbitrarily as we need.
# With the alphabet being our data, we currently have the sequence:
# [a b c d e f g h i j k l m n o p q r s t u v w x y z]
# We want to reorganize it as independant batches that will be processed independantly by the model !
# For instance, with the alphabet as the sequence and batch size 4, we'd get the 4 following sequences:
# ┌ a g m s ┐
# │ b h n t │
# │ c i o u │
# │ d j p v │
# │ e k q w │
# └ f l r x ┘
# with the last two elements being lost.
# Again, these columns are treated as independent by the model, which means that the
# dependence of e. g. 'g' on 'f' can not be learned, but allows more efficient processing.

def batchify(data, batch_size, cuda = False):
    # Cut the elements that are unnecessary
    nbatch = data.size(0) // batch_size
    data = data.narrow(0, 0, nbatch * batch_size)
    # Reorganize the data
    data = data.view(batch_size, -1).t().contiguous()
    # If we can use a GPU, let's tranfer the tensor to it
    return data.to(device)

# get_batch subdivides the source data into chunks of the appropriate length.
# If source is equal to the example output of the batchify function, with
# a sequence length (seq_len) of 3, we'd get the following two variables:
# ┌ a g m s ┐ ┌ b h n t ┐
# | b h n t | | c i o u │
# └ c i o u ┘ └ d j p v ┘
# The first variable contains the letters input to the network, while the second
# contains the one we want the network to predict (b for a, h for g, v for u, etc..)
# Note that despite the name of the function, we are cutting the data in the
# temporal dimension, since we already divided data into batches in the previous
# function. 

def get_batch(source, i, seq_len, evaluation=False):
    # Deal with the possibility that there's not enough data left for a full sequence
    seq_len = min(seq_len, len(source) - 1 - i)
    # Take the input data
    data = source[i:i+seq_len]
    # Shift by one for the target data
    target = source[i+1:i+1+seq_len]
    return data, target

In [74]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [75]:
batch_size = 100
eval_batch_size = 4
train_data = batchify(corpus.train, batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

print(train_data.shape)
print(val_data.shape)

torch.Size([15187, 100])
torch.Size([91601, 4])


### LSTM Cells in pytorch

### Creating our own LSTM Model

In [76]:
# Models are usually implemented as custom nn.Module subclass
# We need to redefine the __init__ method, which creates the object
# We also need to redefine the forward method, which transform the input into outputs
# We can also add any method that we need: here, in order to initiate weights in the model

class LSTMModel(nn.Module):
    def __init__(self, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(LSTMModel, self).__init__()
        # Create a dropout object to use on layers for regularization
        self.drop = nn.Dropout(dropout)
        # Create an encoder - which is an embedding layer
        self.encoder = nn.Embedding(ntoken, ninp)
        # Create the LSTM layers - find out how to stack them !
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        # Create what we call the decoder: a linear transformation to map the hidden state into scores for all words in the vocabulary
        # (Note that the softmax application function will be applied out of the model)
        self.decoder = nn.Linear(nhid, ntoken)
        
        # Initialize non-reccurent weights 
        self.init_weights()

        self.ninp = ninp
        self.nhid = nhid
        self.nlayers = nlayers
        
    def init_weights(self):
        # Initialize the encoder and decoder weights with the uniform distribution
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)
        
    def init_hidden(self, batch_size):
        # Initialize the hidden state and cell state to zero, with the right sizes
        weight = next(self.parameters())
        return (weight.new_zeros(self.nlayers, batch_size, self.nhid),
                weight.new_zeros(self.nlayers, batch_size, self.nhid))    

    def forward(self, input, hidden, return_h=False):
        # Process the input
        emb = self.drop(self.encoder(input))   
        
        # Apply the LSTMs
        output, hidden = self.rnn(emb, hidden)
        
        # Decode into scores
        output = self.drop(output)      
        decoded = self.decoder(output)
        return decoded, hidden

### Building the Model

In [77]:
# Set the random seed manually for reproducibility.
torch.manual_seed(1)

In [78]:
embedding_size = 500
hidden_size = 500
layers = 2
dropout = 0.3

###############################################################################
# Build the model
###############################################################################

vocab_size = len(corpus.dictionary)
model = LSTMModel(vocab_size, embedding_size, hidden_size, layers, dropout).to(device)
params = list(model.parameters())
criterion = nn.CrossEntropyLoss()

In [79]:
lr = 10.0
optimizer = 'sgd'
wdecay = 1.2e-6
# For gradient clipping
clip = 0.25

if optimizer == 'sgd':
    optim = torch.optim.SGD(params, lr=lr, weight_decay=wdecay)
if optimizer == 'adam':
    optim = torch.optim.Adam(params, lr=lr, weight_decay=wdecay)

In [80]:
# Let's think about gradient propagation:
# We plan to keep the second ouput of the LSTM layer (the hidden/cell states) to initialize
# the next call to LSTM. In this way, we can back-propagate the gradient for as long as we want.
# However, this put a huge strain on the memory used by the model, since it implies retaining
# a always-growing number of tensors of gradients in the cache.
# We decide to not backpropagate through time beyond the current sequence ! 
# We use a specific function to cut the 'hidden/state cell' states from their previous dependencies
# before using them to initialize the next call to the LSTM.
# This is done with the .detach() function.

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [81]:
# Other global parameters
epochs = 50
seq_len = 30
log_interval = 10
save = 'model.pt'

In [82]:
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, seq_len):
            data, targets = get_batch(data_source, i, seq_len)
            output, hidden = model(data, hidden)
            hidden = repackage_hidden(hidden)
            total_loss += len(data) * criterion(output.view(-1, vocab_size), targets.view(-1)).item()
    return total_loss / (len(data_source) - 1)

In [83]:
def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    hidden = model.init_hidden(batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, seq_len)):
        data, targets = get_batch(train_data, i, seq_len)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        optim.zero_grad()
        
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, vocab_size), targets.view(-1))
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(params, clip)
        optim.step()
        
        total_loss += loss.data

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // seq_len, lr,
                elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [84]:
# Loop over epochs.
best_val_loss = None

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open(save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

# Load the best saved model.
with open(save, 'rb') as f:
    model = torch.load(f)
    # after load the rnn params are not a continuous chunk of memory
    # this makes them a continuous chunk, and will speed up forward pass
    model.rnn.flatten_parameters()

# Run on test data.
test_loss = evaluate(test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| epoch   1 |    10/  506 batches | lr 10.00 | ms/batch 136.57 | loss 10.65 | ppl 42175.68
| epoch   1 |    20/  506 batches | lr 10.00 | ms/batch 118.57 | loss  8.33 | ppl  4159.53
| epoch   1 |    30/  506 batches | lr 10.00 | ms/batch 119.70 | loss  7.83 | ppl  2518.34
| epoch   1 |    40/  506 batches | lr 10.00 | ms/batch 118.45 | loss  7.39 | ppl  1625.37
| epoch   1 |    50/  506 batches | lr 10.00 | ms/batch 118.47 | loss  7.29 | ppl  1471.76
| epoch   1 |    60/  506 batches | lr 10.00 | ms/batch 117.92 | loss  7.16 | ppl  1284.64
| epoch   1 |    70/  506 batches | lr 10.00 | ms/batch 118.93 | loss  7.06 | ppl  1167.76
| epoch   1 |    80/  506 batches | lr 10.00 | ms/batch 119.33 | loss  6.99 | ppl  1083.31
| epoch   1 |    90/  506 batches | lr 10.00 | ms/batch 119.73 | loss  6.96 | ppl  1055.49
| epoch   1 |   100/  506 batches | lr 10.00 | ms/batch 119.64 | loss  6.80 | ppl   901.43
| epoch   1 |   110/  506 batches | lr 10.00 | ms/batch 119.76 | loss  6.71 | ppl   817.03

In [85]:
def beam_search_decode(device, net, words, vocab_to_int, int_to_vocab, top_k, temperature):
  net.eval()
  softmax = nn.Softmax(dim = 1)
  words = words.split(' ')
  words.append('<sos>')
  hidden = net.init_hidden(1)
  for v in hidden:
    v = v.to(device)
  for w in words:
    ix = torch.tensor([[vocab_to_int[w]]]).to(device)
    output, hidden = net(ix, hidden)
  output = output / temperature
  prob, top_ix = torch.topk(softmax(output[0]), k=top_k)
  prob = torch.log(prob)
  #print(top_ix)
  list_ids = [[id] for id in top_ix[0].tolist()]
  outputs = [output for _ in range(top_k)]
  hiddens = [hidden for _ in range(top_k)] 
  #print("avant beam search, top indices : ",top_ix.tolist(), "de proba", prob.tolist())
  #print(list_ids)
  for i in range(40):
    probas = torch.zeros(top_k, top_k).float().to(device)
    indxes = torch.zeros(top_k, top_k).to(device)
    for k in range(top_k):
      ix = torch.tensor([[top_ix[0][k]]]).to(device)
      output, hiddens[k] = net(ix, hiddens[k])
      output = output / temperature
      pro, indxes[k] = torch.topk(softmax(output[0]), k=top_k)
      pro = torch.log(pro)
      #print("probas du choix ", k+1," : ", pro.tolist())
      probas[k] = torch.add(pro[0], prob[0][k])
    #print(indxes.tolist())
    #print(list_ids)
    prob, indices = torch.topk(probas.flatten(), top_k)
    prob = torch.unsqueeze(prob, 0)
    for k in range(top_k):
      top_ix[0][k] = indxes.flatten()[indices[k]]
    indices = indices // top_k
    temp1 = []
    temp2 = []
    for k in range(top_k):
      temp1.append(hiddens[indices.tolist()[k]])
      temp2.append(list_ids[indices.tolist()[k]] + [top_ix[0].tolist()[k]])
    hiddens = temp1
    list_ids = temp2
    #print("top indices : ",top_ix.tolist(), "de proba", prob.tolist())
  best_branch = list_ids[torch.argmax(prob)]
  words = []
  for id in best_branch:
    words.append(int_to_vocab[id])
  print(' '.join(words))

In [86]:
def predict(device, net, words, vocab_to_int, int_to_vocab, temperature):
  net.eval()
  softmax = nn.Softmax(dim=1)
  words = words.split(' ')
  hidden = net.init_hidden(1)
  for v in hidden:
    v = v.to(device)
  for w in words:
    ix = torch.tensor([[vocab_to_int[w]]]).to(device)
    output, hidden = net(ix, hidden)
  output = output / temperature
  idx_max = torch.argmax(softmax(output[0]))
  words = []
  words.append(int_to_vocab[idx_max])
  for i in range(0, 40):
      ix = torch.tensor([[idx_max]]).to(device)
      output, hidden = net(ix, hidden)
      output = output / temperature
      idx_max = torch.argmax(softmax(output[0]))
      words.append(int_to_vocab[idx_max])
  print(' '.join(words))

In [87]:
def top_k_sampling(device, net, words, vocab_to_int, int_to_vocab, top_k, temperature):
  net.eval()
  softmax = nn.Softmax(dim=-1)
  words = words.split(' ')
  hidden = net.init_hidden(1)
  for v in hidden:
    v = v.to(device)
  for w in words:
    ix = torch.tensor([[vocab_to_int[w]]]).to(device)
    output, hidden = net(ix, hidden)
  output = output / temperature
  indices_to_remove = output[0] < torch.topk(output[0], top_k)[0][..., -1, None]
  output[0][indices_to_remove] = -float('Inf')
  prob = softmax(output[0])
  idx_max = torch.multinomial(prob, 1)
  words = []
  words.append(int_to_vocab[idx_max])
  for i in range(0, 40):
      ix = torch.tensor([[idx_max]]).to(device)
      output, hidden = net(ix, hidden)
      output = output[0][0] / temperature
      indices_to_remove = output < torch.topk(output, top_k)[0][..., -1, None]
      output[indices_to_remove] = -float('Inf')
      prob = softmax(output)
      idx_max = torch.multinomial(prob, 1)
      words.append(int_to_vocab[idx_max])
  print(' '.join(words))

In [88]:
words = '<sos> Mettant au plus heureux le sceptre dans la main , <eos1> <sos> Va faire l’ un sujet , et l’ autre souverain . <eos2>'
predict(device, model, words, corpus.dictionary.word2idx, corpus.dictionary.idx2word,1)
beam_search_decode(device, model, words, corpus.dictionary.word2idx, corpus.dictionary.idx2word,10,0.2)
beam_search_decode(device, model, words, corpus.dictionary.word2idx, corpus.dictionary.idx2word,10,0.5)
beam_search_decode(device, model, words, corpus.dictionary.word2idx, corpus.dictionary.idx2word,10,1)
top_k_sampling(device, model, words, corpus.dictionary.word2idx, corpus.dictionary.idx2word, 10, 1)
top_k_sampling(device, model, words, corpus.dictionary.word2idx, corpus.dictionary.idx2word, 50, 1)
top_k_sampling(device, model, words, corpus.dictionary.word2idx, corpus.dictionary.idx2word, 50, 0.5)
top_k_sampling(device, model, words, corpus.dictionary.word2idx, corpus.dictionary.idx2word, 50, 0.2)

<sos> Mais , Madame , voyez que je suis votre mère . <eos1> <sos> Vous en êtes , Madame , et pour vous en parler , <eos2> <sos> Et vous pouvez , seigneur , de ne le point souffrir . <eos1>
Mais , Madame , voyez que je suis à vous - même . <eos1> <sos> Je vous l’ ai déjà dit , je vous en fais justice . <eos2> <sos> Je vous l’ ai déjà dit , je vous en fais
Il faut que je le voie , et que je te pardonne . <eos1> <sos> Mais , Madame , après tout , que faut - il que je fasse ? <eos2> <sos> Que ne puis - je , Seigneur , que
Ce n’ est qu’ un faux effet qu’ il faut que je lui fasse . <eos1> <sos> Mais , Madame , après tout , je n’ ai rien à vous dire . <eos2> <sos> Je ne vous dirai point que je
<sos> Vous avez en ces lieux quelque chose d’ un père . <eos1> <sos> J’ en réponds , j’ y consens , mais je l’ y vois paraître . <eos2> <sos> Que me vient de l’ amour ! Et qu’ en
<sos> Qu’ elle parte , elle fuit , et que Rome il vit . <eos1> <sos> Je voudrais , à sa vue , en sa protection , <eos2> <sos>

Je veux , et l’ en a vu . . Mais il faut à l’ entendre : . . ; <eos> 

Je vous dois voir à vous . Et que j’ ai su l’ attendre <eos> 

Ce que j’ avais pu voir , et l’ autre à l’ honneur ; <eos> 

Il n’ a rien dit qu’ en ce mot je n’ en dois point de rien

**Ajout du SoS**

. . . Ah ! Seigneur , c’ est moi - même . <eos> 

<sos> Ah ! Madame , il est vrai , je n’ en veux point douter , <eos>

<sos> Et je n’ ai pas besoin de m’ en faire haïr .